In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import numpy as np
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

otu_table = pd.read_csv('genus_rotated_f_filtered.csv', header=0, index_col=None)

scaler = StandardScaler()
otu_table_scaled = scaler.fit_transform(otu_table)

otu_table_scaled_df = pd.DataFrame(otu_table_scaled, columns=otu_table.columns)


genus_names = otu_table_scaled_df.columns.tolist()
genus_to_idx = {genus: idx for idx, genus in enumerate(genus_names)}


otu_table_scaled_df

In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import numpy as np
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

otu_table = pd.read_csv('genus_rotated_f_filtered.csv', header=0, index_col=None)

# 将每一行转换为百分比
otu_percentage = otu_table.div(otu_table.sum(axis=1), axis=0) * 100

# 设定分箱的数量，例如 5 个箱
num_bins = 50

otu_discrete = otu_percentage.apply(lambda x: pd.cut(x, bins=num_bins, labels=range(1, num_bins + 1)), axis=1)
otu_discrete = otu_discrete.apply(pd.to_numeric)
otu_table_scaled_df = pd.DataFrame(otu_discrete, columns=otu_table.columns)


genus_names = otu_table_scaled_df.columns.tolist()
genus_to_idx = {genus: idx for idx, genus in enumerate(genus_names)}


otu_table_scaled_df

,Akkermansia,Alistipes,Bacteroides,Blautia,Clostridium,Collinsella,Coprococcus,Dialister,Dorea,Eubacterium,...,Lachnospira,Odoribacter,Oscillospira,Parabacteroides,Phascolarctobacterium,Prevotella,Roseburia,Ruminococcus,Subdoligranulum,Sutterella
0,1,3,50,1,1,1,1,1,1,1,...,1,1,1,3,1,1,1,1,1,2
1,1,1,50,2,1,1,1,1,1,2,...,1,1,1,8,1,1,2,1,2,1
2,1,1,50,1,1,1,1,1,1,1,...,1,1,1,2,1,1,2,1,1,1
3,3,25,50,2,3,1,2,1,1,2,...,1,1,16,4,1,1,20,8,9,1
4,1,6,50,3,3,1,1,1,1,1,...,1,1,6,25,5,1,11,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,1,1,50,1,1,1,1,1,1,1,...,1,1,1,1,1,1,2,1,1,3
315,1,1,50,1,1,1,1,1,1,1,...,1,1,4,1,1,1,1,1,1,1
316,1,9,50,3,8,1,2,7,1,1,...,1,1,15,1,14,11,2,13,9,1
317,1,1,23,1,2,1,3,9,1,1,...,1,1,5,7,1,50,1,3,1,1


In [5]:
# torch.cuda.empty_cache()
class OTUDataset(Dataset):# class token
    def __init__(self, csv, mask_prob=0.15):
        self.data = csv
        self.mask_prob = mask_prob
        self.data_tensor = torch.tensor(self.data.values, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data_tensor[idx]
        masked_sample, mask = self.mask_data(sample)
        return masked_sample, sample, mask

    def mask_data(self, sample):
        mask = torch.bernoulli(torch.full(sample.shape, self.mask_prob)).bool()
        masked_sample = sample.clone()
        masked_sample[mask] = 0  # Using 0 as a placeholder for masked values
        return masked_sample, mask


# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class TransformerEncoderModel(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_layers):
        super(TransformerEncoderModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model) # include mask + 1
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.fc = nn.Linear(d_model, vocab_size)# y(i) = W*x + b, i belong to range(vocab_size)

    def forward(self, x, return_features=False):
        x= x.long()
        x = self.embedding(x)
        x = self.transformer_encoder(x.permute(1, 0, 2))
        x = x.permute(1, 0, 2)
        if return_features:
            return x
        x = self.fc(x)
        print(x.shape)
        return x

vocab_size = len(genus_names)
d_model = 64
nhead = 8
num_layers = 6
# print(vocab_size)

model = TransformerEncoderModel(vocab_size, d_model, nhead, num_layers).to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create a dataset and dataloader
dataset = OTUDataset(otu_table_scaled_df)
# print(dataset)
data_loader = DataLoader(dataset, batch_size=1, shuffle=True)
# for  masked_data, original_data, mask in data_loader:
#     masked_data, original_data, mask = masked_data.to(device), original_data.to(device), mask.to(device)
#     outputs = model(masked_data)
#     print("masked_data: ",masked_data[mask])
#     print("original_data: ",original_data[mask])
#     print("mask: ",mask)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

In [3]:
'''
d_model = 4
output = [
    [2,2,4,5]   #token 0
    [1,3,4,5]   #token 1
    [4,1,3,6]   #token 2
]
'''
def train(model, data_loader, criterion, optimizer, device, epochs=32):
    model.train()
    losses = []
    for epoch in range(epochs):
        total_loss = 0
        for masked_data, original_data, mask in data_loader:
            masked_data, original_data, mask = masked_data.to(device), original_data.to(device), mask.to(device)
            optimizer.zero_grad()
            outputs = model(masked_data)
            # Flatten tensors for loss calculation
            masked_outputs = outputs[mask]
            masked_targets = original_data[mask].long()
            loss = criterion(masked_outputs, masked_targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(data_loader)
        losses.append(avg_loss)
        print(f'Epoch {epoch+1}/{epochs}, Loss: {avg_loss}')
    return losses
torch.cuda.empty_cache()
losses = train(model, data_loader, criterion, optimizer, device)


torch.save(model.state_dict(), 'pretrained_transformer_encoder_model.pth')


plt.figure(figsize=(10, 5))
plt.plot(range(1, len(losses) + 1), losses, marker='o')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.grid(True)
plt.show()

/opt/conda/conda-bld/pytorch_1678411187366/work/aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [1,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1678411187366/work/aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [1,0,0], thread: [1,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1678411187366/work/aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [1,0,0], thread: [2,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1678411187366/work/aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [1,0,0], thread: [3,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1678411187366/work/aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [1,0,0], thread: [4,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pyto

RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`

In [2]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import numpy as np
import torch.optim as optim

# Set device
device = torch.device( "cpu")

# Load and preprocess the data
otu_table = pd.read_csv('genus_rotated_f_filtered.csv', header=0, index_col=None)

# Convert each row to percentage
otu_percentage = otu_table.div(otu_table.sum(axis=1), axis=0) * 100

# Set the number of bins
num_bins = 50

# Discretize the data
otu_discrete = otu_percentage.apply(lambda x: pd.cut(x, bins=num_bins, labels=range(1, num_bins + 1)), axis=1)
otu_discrete = otu_discrete.apply(pd.to_numeric)
otu_table_scaled_df = pd.DataFrame(otu_discrete, columns=otu_table.columns)

# Generate genus to index mapping
genus_names = otu_table_scaled_df.columns.tolist()
genus_to_idx = {genus: idx for idx, genus in enumerate(genus_names)}

# Define the dataset class
class OTUDataset(Dataset):
    def __init__(self, csv, mask_prob=0.15):
        self.data = csv
        self.mask_prob = mask_prob
        self.data_tensor = torch.tensor(self.data.values, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data_tensor[idx]
        masked_sample, mask = self.mask_data(sample)
        return masked_sample, sample, mask

    def mask_data(self, sample):
        mask = torch.bernoulli(torch.full(sample.shape, self.mask_prob)).bool()
        masked_sample = sample.clone()
        masked_sample[mask] = 0  # Using 0 as a placeholder for masked values
        return masked_sample, mask

# Define the Transformer model
class TransformerEncoderModel(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_layers):
        super(TransformerEncoderModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size + 1, d_model)  # Include mask + 1
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.fc = nn.Linear(d_model, vocab_size)  # y(i) = W*x + b, i belong to range(vocab_size)

    def forward(self, x, return_features=False):
        x = x.long()
        x = self.embedding(x)
        x = self.transformer_encoder(x.permute(1, 0, 2))
        x = x.permute(1, 0, 2)
        if return_features:
            return x
        x = self.fc(x)
        # print(x.shape)
        return x

# Set parameters for the model
vocab_size = len(genus_names)
d_model = 64
nhead = 8
num_layers = 6

# Instantiate the model, criterion, and optimizer
model = TransformerEncoderModel(vocab_size, d_model, nhead, num_layers).to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create a dataset and dataloader
dataset = OTUDataset(otu_table_scaled_df)
data_loader = DataLoader(dataset, batch_size=1, shuffle=True)

# Training loop
num_epochs = 10
# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for masked_sample, original_sample, mask in data_loader:
        masked_sample, original_sample, mask = masked_sample.to(device), original_sample.to(device), mask.to(device)
        optimizer.zero_grad()
        output = model(masked_sample)
        output = output.view(-1, vocab_size)
        original_sample = original_sample.view(-1)
        mask = mask.view(-1)
        loss = criterion(output[mask], original_sample[mask].long())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(data_loader)}")


IndexError: index out of range in self

In [1]:
import torch.nn as nn
class EnhancedMLP(nn.Module):
    def __init__(self, pretrained_model, input_size, hidden_size, num_classes):
        super(EnhancedMLP, self).__init__()
        self.pretrained_model = pretrained_model
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    # def forward(self, x):
    #     with torch.no_grad():
    #         x = self.pretrained_model(x.long(), return_features=True)  # 获取编码后的特征
    #     x = x.mean(dim=1)  # 对序列维度进行平均池化
    #     x = torch.nn.functional.relu(self.fc1(x))
    #     x = self.fc2(x)
    #     return x
    def forward(self, x):
        x = self.pretrained_model(x, return_features=True)
        x = torch.nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [2]:
# 加载预训练的TransformerEncoderModel
pretrained_model = TransformerEncoderModel(22, d_model, nhead, num_layers)
pretrained_model.load_state_dict(torch.load('pretrained_transformer_encoder_model.pth'))
pretrained_model.to(device)
pretrained_model.eval()  # 设置为评估模式

# EnhancedMLP模型参数
input_size = vocab_size * d_model  # 根据TransformerEncoderModel的输出尺寸调整
hidden_size = 128
num_classes = 10  # 假设有10个类别进行分类

# 初始化EnhancedMLP模型并移动到GPU
enhanced_mlp = EnhancedMLP(pretrained_model, input_size, hidden_size, num_classes).to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = Adam(enhanced_mlp.parameters(), lr=0.0001)
enhanced_mlp

NameError: name 'TransformerEncoderModel' is not defined

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader

# Read the data into a DataFrame
df = pd.read_csv("NSCLC.csv")
# Filter out columns that contain all zeros
# df = df.loc[:, (NSCLC != 0).any(axis=0)]
f1 = pd.read_csv('genus_rotated_f_filtered.csv')
# print(f1.shape)
# Extract genus-level data
df['Genus'] = df['#NAME'].apply(lambda x: x.split(';g__')[1].split(';')[0] if ';g__' in x else 'Unclassified')

# Select the relevant columns
genus_df = df[['Genus'] + df.columns[1:-1].tolist()]

# Filter out rows with "_unclassified" in the Genus column
NSCLC = genus_df[~genus_df['Genus'].str.contains('_unclassified')]
if 'Genus' in NSCLC.columns:
    NSCLC = NSCLC.groupby('Genus').sum().reset_index()
else:
    NSCLC = NSCLC  # If there is no 'genus' column, use the original data
NSCLC = NSCLC[NSCLC['Genus'].notna() & (NSCLC['Genus'] != '')]
NSCLC = NSCLC.loc[:, (NSCLC != 0).any(axis=0)]
NSCLC.set_index(NSCLC.columns[0], inplace=True)
f2 = NSCLC.transpose()
missing_cols = [col for col in f1.columns if col not in f2.columns]
# Add missing columns to f2 with values set to 0 using pd.concat
f2 = pd.concat([f2, pd.DataFrame(0, index=f2.index, columns=missing_cols)], axis=1)
# Drop columns from f2 that are not in f1
f2 = f2[f1.columns]
# Merge f2 to f1, keeping only the column names
f1 = f2
metadata  = pd.read_csv('metadata_response.csv')
metadata.set_index(metadata.columns[0], inplace=True)
# num_columns = len(merged_table.columns) - 1
merged_table = f1.join(metadata, how='inner')
# merged_table.to_csv("merged_table.csv",index=False)
# merged_table = merged_table.drop(columns=['Best response'])
response = merged_table['Best response']
otu_table_merge = merged_table.drop(columns=['Best response'])
# Drop the first column if it contains sample IDs or unnecessary data
otu_table_merge = otu_table_merge.iloc[:, 1:]

# # Normalize OTU counts by total counts per sample
# normalized_otu_counts = otu_table_merge.div(otu_table_merge.sum(axis=1), axis=0)

# # Optionally, convert to percentages
# normalized_otu_counts *= 100
# 标准化OTU数据
scaler = StandardScaler()
otu_table_scaled = scaler.fit_transform(otu_table_merge)
# normalized_otu_counts = otu_table_merge
# 将标准化后的数据转换回DataFrame
normalized_otu_counts = pd.DataFrame(otu_table_scaled, columns=otu_table_merge.columns)
# Print to verify
# normalized_otu_counts.to_csv("normalized_otu_counts.csv",index=False)
# Create a dictionary to map genus names to unique indices
genus_names = normalized_otu_counts.columns.tolist()
genus_to_idx = {genus: idx for idx, genus in enumerate(genus_names)}
genus_names
genus_to_idx
from sklearn.preprocessing import LabelEncoder

# Assuming 'merged_table' is your DataFrame containing the response and features
encoder = LabelEncoder()
merged_table['Best response'] = encoder.fit_transform(merged_table['Best response'])

# Separate features and target
# features = merged_table.drop('Best response', axis=1)
features = normalized_otu_counts
targets = merged_table['Best response']
targets

In [ ]:
class OTUDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        # Ensure data is returned as tensors
        x = torch.tensor(self.features.iloc[idx].values, dtype=torch.float32)
        y = torch.tensor(self.targets.iloc[idx], dtype=torch.long)  # Use torch.long for classification labels
        return x, y
from sklearn.model_selection import train_test_split
# Encode the 'Best response' column
encoder = LabelEncoder()
merged_table['Best response'] = encoder.fit_transform(merged_table['Best response'])

# Split the features and targets into training and testing sets
features_train, features_test, targets_train, targets_test = train_test_split(
    features, targets, test_size=0.2, random_state=42)
# print(features_test)
train_dataset = OTUDataset(features_train, targets_train)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataset = OTUDataset(features_test, targets_test)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [ ]:
# Train and evaluate the enhanced model
def train_and_evaluate(model, train_dataloader, test_dataloader, criterion, optimizer, num_epochs=256):
    # Check if GPU is available and move the model to GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    def train_model(model, criterion, optimizer, dataloader, num_epochs):
        model.train()
        for epoch in range(num_epochs):
            total_loss = 0
            for inputs, labels in dataloader:
                # Move inputs and labels to GPU
                inputs, labels = inputs.to(device), labels.to(device)
                
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            if (epoch + 1) % 10 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(dataloader):.4f}')

    def evaluate_model(model, dataloader):
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in dataloader:
                # Move inputs and labels to GPU
                inputs, labels = inputs.to(device), labels.to(device)
                
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        print(f'Accuracy: {accuracy:.2f}%')

    # Train the model
    train_model(model, criterion, optimizer, train_dataloader,device,num_epochs)
    
    # Evaluate the model
    evaluate_model(model, test_dataloader)



In [ ]:
hidden_size = 128
enhanced_model = EnhancedMLP(pretrained_model, d_model, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(enhanced_model.parameters(), lr=0.001)

# Train and evaluate the enhanced model
train_and_evaluate(enhanced_model, train_dataloader, test_dataloader, criterion, optimizer,num_epochs=64)